# QAMaster: Fine-tuned Q/A Chat Box
- Summer 2024

This project creates a Q/A chat box using multiple pre-trained models. It fine-tunes these models on a Q/A dataset and evaluates them to choose the best one based on F1-score and Exact Match.

hf access token:

In [1]:
token = ''

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, AutoConfig
from datasets import load_dataset, Dataset
import numpy as np
from sklearn.model_selection import train_test_split

## Intro
- Loading Pre-trained Models
- Loading Squad dataset

In [8]:
models_to_finetune = [
    "Qwen/Qwen1.5-72B",     # QWEn 1.5.72B
    "tuner007/pegasus_paraphrase",  # Scientific Research
    "gpt2",                 # gpt2
    "deepset/roberta-base-squad2",  # Example model 1 (RoBERTa fine-tuned on SQuAD 2.0)
    "neuralmind/bert-large-portuguese-cased"  # Example model 2 (BERT large for Portuguese)
]

tokenizer_models = {}
for model_name in models_to_finetune:
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=token)
    config = AutoConfig.from_pretrained(model_name, use_auth_token=token)
    model = AutoModelForCausalLM.from_pretrained(model_name, config=config, use_auth_token=token)
    
    tokenizer_models[model_name] = (tokenizer, model)



tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

/Users/salehbabaei/opt/anaconda3/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

ValueError: Couldn't instantiate the backend tokenizer from one of: 
(1) a `tokenizers` library serialization file, 
(2) a slow tokenizer instance to convert or 
(3) an equivalent slow tokenizer class to instantiate and convert. 
You need to have sentencepiece installed to convert a slow tokenizer to a fast one.

In [ ]:
def load_qa_dataset():
    dataset = load_dataset("squad")
    return dataset
dataset = load_qa_dataset()

## PreProcessing
- Split QA Dataset into Train, Validation, and Test Sets

In [ ]:
train_valid_dataset = dataset["train"]
test_dataset = dataset["validation"]

train_dataset, eval_dataset = train_test_split(train_valid_dataset, test_size=0.2, random_state=42)

## Fine-tuning Function
- Fine-tuning function defined

In [ ]:
def fine_tune_model(model_name, tokenizer, model, train_dataset, eval_dataset, training_args):
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
    )

    trainer.train()

    eval_results = trainer.evaluate()

    return eval_results

## Fine-tune and Evaluation
- Training arguments
- Results
- Best Model

In [ ]:
training_args = TrainingArguments(
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    logging_dir="./logs",
    logging_steps=100,
)

In [ ]:
results = {}

for model_name, (tokenizer, model) in tokenizer_models.items():
    print(f"Fine-tuning {model_name}...")
    eval_results = fine_tune_model(model_name, tokenizer, model, train_dataset, eval_dataset, training_args)
    results[model_name] = eval_results

    print(f"Evaluation results for {model_name}:")
    print(f"Exact Match: {eval_results['exact_match']:.2f}")
    print(f"F1 Score: {eval_results['f1']:.2f}")
    print()

In [ ]:
best_model = max(results, key=lambda x: results[x]['f1'])
print(f"Best model based on F1 score: {best_model}")
print(f"Evaluation results - Exact Match: {results[best_model]['exact_match']:.2f}, F1 Score: {results[best_model]['f1']:.2f}")